* You should run Google_OAuth2_Save_Credentials.ipynb and save user's credentials ('oauth2_user_credentials.json') first
### Docs
* https://developers.google.com/youtube/v3/docs

In [ ]:
# -*- coding: utf-8 -*-

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  

import os, json, flask, requests

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
from googleapiclient.http import MediaFileUpload

def revoke_credentials(): 
    revoke = requests.post('https://oauth2.googleapis.com/revoke',
        params={'token': credentials.token},
        headers = {'content-type': 'application/x-www-form-urlencoded'})

    status_code = getattr(revoke, 'status_code')
    if status_code == 200:
        return('Credentials successfully revoked.')
    else:
        return('An error occurred.') 

In [ ]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
with open('oauth2_user_credentials.json') as fr:
    credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr))
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

#### List playlists in my youtube channel

In [ ]:
playlists = youtube.playlists().list(part='snippet', mine=True).execute()
[{'playlistId':i['id'], 'title':i['snippet']['title']} for i in playlists['items']]

#### Another way to list my playlists

In [ ]:
channel   = youtube.channels().list(mine=True, part='id').execute() 
channelId = channel['items'][0]['id']
print('My channel id:', channelId)

playlists = youtube.playlists().list(part='snippet',channelId=channelId).execute()
print([{'playlistId':i['id'], 'title':i['snippet']['title']} for i in playlists['items']])
'''
My channel id: UCAA_xxxxxgS05YLF-xxxx-Q
[{'playlistId': 'PLxxxxxxxek07', 'title': 'test'},
 {'playlistId': 'PLxxxxxxxmViZ', 'title': '123'},
 {'playlistId': 'PLxxxxxxxmTwd', 'title': '456'}]
'''

#### What is in one playlist

In [ ]:
playlistItems = youtube.playlistItems().list(
    playlistId='PLxxxxxxxmViZ',
    part='snippet',
    maxResults=5
  ).execute() 
[{'title':i['snippet']['title'], 'videoId':i['snippet']['resourceId']['videoId']} for i in playlistItems['items']]

#### Upload a video

In [ ]:
body={
    'snippet':{
        'title':'Test The Upload API',
        'description':'More words for this video',
        'tags':['a','b'],
        'categoryId':'22' # https://developers.google.com/youtube/v3/docs/videoCategories/list
    },
    'status':{
        'privacyStatus':'unlisted', # ['private','public','unlisted']
        'selfDeclaredMadeForKids':False
    }
}
uploadedVideo = youtube.videos().insert(
    part=','.join(body.keys()),
    body=body, 
    media_body=MediaFileUpload('test.mkv', chunksize=-1, resumable=False)
).execute()
videoId   = uploadedVideo['id']
print(videoId)

#### Add video to a playlist

In [ ]:
body = {'snippet':{
    "playlistId": playlistId,
    #"position": 0,
    "resourceId": {
      "kind": "youtube#video",
      "videoId": videoId
    }
}}
resource = youtube.playlistItems().insert(part='snippet', body=body).execute()

#### Help

In [ ]:
_ = youtube.playlistItems()
?_.insert